# Introduction

Portfolio management is a crucial field in finance. It involves not only selecting the right investments but also understanding how to combine them optimally to achieve an ideal balance between risk and return. One of the most famous and influential models in this field is the Modern Portfolio Theory (MPT), introduced by Harry Markowitz in 1952.

The fundamental principle of MPT is that investors seek to maximize their return for a given level of risk. This is where the concept of diversification comes into play - by combining different assets with imperfectly correlated returns, one can achieve a more stable overall performance and thus reduce risk.

The portfolio variance, which measures the variability of its returns, is often used as a risk measure in MPT. By minimizing the variance, we aim to make the portfolio returns as stable and predictable as possible.

In this notebook, we will use MPT to optimize an investment portfolio. We will employ optimization techniques to determine the optimal allocation of assets that minimizes the portfolio variance. The weights we obtain will be used to enhance our trading strategy.

We will go through several steps to accomplish this:

- **Data collection**: We will use historical data on asset returns in our portfolio.

- **Calculating returns and covariance**: We will use this data to estimate expected returns and the covariance of asset returns.

- **Portfolio optimization**: We will use an optimization algorithm to determine the weights of each asset in the portfolio that minimizes the variance.

By following these steps, we will obtain an optimized investment strategy that can help us make more informed decisions and, hopefully, achieve better returns.

So let's get started without further ado!

In [124]:
import pandas as pd
import numpy as np
import sqlite3
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import skew, kurtosis, norm, spearmanr, pearsonr
from scipy.optimize import minimize
import MetaTrader5 as mt5
import sys
sys.path.append(r'C:\Users\ftiag\Desktop\Business, trading et investissement\Business\modules')
from research_tools import get_clean_mt5_data

mt5.initialize()

login_mt5 = 1051534030
mdp_mt5 = 'FG2SF2M74R'
server = 'FTMO-Demo'

mt5.login(login_mt5, mdp_mt5, server)

True

In this part of the code, we start by defining the assets of our portfolio. These are the symbols of the currency pairs that we are going to analyze. Using the identifiers of these pairs ('GBPJPY', 'GBPUSD', 'EURGBP', 'EURUSD', 'USDCAD', 'USDJPY'), we can obtain relevant market information for our analysis.

We then use a function called `get_clean_mt5_data` to retrieve data for these assets. This function is part of a personal module that I created to facilitate market data analysis. It uses the MetaTrader 5 (MT5) API to retrieve market data and returns a "cleaned" version of this data.

The `interval` parameter specifies the time interval for the data we retrieve. Here, we are using `mt5.TIMEFRAME_H4`, which means that we are retrieving data on a 4-hour interval.

The `n_bars` parameter specifies the number of bars (or periods) that we want to retrieve. Here, we are using `5000`, which means that we are retrieving data for the last 5000 periods.

Finally, the `.close` at the end of the function call means that we are only interested in the closing prices for each period.

So, here's how we can interpret this line of code:

"For each currency pair in our ticker list, retrieve the last 5000 closing prices on 4-hour intervals using the `get_clean_mt5_data` function from our personal module."

In [125]:
tickers = ['GBPJPY', 'GBPUSD', 'EURGBP', 'EURUSD', 'USDCAD', 'USDJPY']
prices = get_clean_mt5_data(tickers, interval=mt5.TIMEFRAME_H4, n_bars=5000).close

Now that we have retrieved our price data, we need to prepare it for our analysis.

The first line `prices = prices.unstack(0).fillna(method='ffill')` reformats the price data into a more suitable structure for our analysis. The `unstack(0)` operation rearranges the DataFrame so that each column corresponds to a currency pair. `fillna(method='ffill')` fills in missing values with the last known valid value, ensuring there are no gaps in our data.

Next, we initialize our weights with a uniform distribution, meaning we initially assume each asset in our portfolio has the same weight. We use the expression `np.array([1/len(returns.columns)]*len(returns.columns))` for this.

We then define the `minimize_variance` function. This function takes a set of weights and calculates the resulting portfolio variance. Returns are computed using `prices.pct_change().dropna()`, which calculates the percentage change in prices from one period to another. Next, we calculate the covariance matrix of returns with `returns.cov()`, which gives us a measure of how the returns of different assets move together. Portfolio variance is calculated using `np.dot(np.dot(weights.T, cov), weights)`, which is a standard formulation for calculating the variance of a portfolio based on asset weights and their covariance matrix.

We also define a constraint on the weights in the `constraint1` function. This constraint ensures that the sum of weights is equal to 1, meaning we are using the entire capital to invest.

Then, we use the `minimize` function from the `scipy.optimize` optimization module to find the weights that minimize the portfolio variance. The `SLSQP` (Sequential Least Squares Programming) method is used as the optimization method. The constraints and weight bounds are also passed to the `minimize` function.

The `result` variable will contain the optimization results, including the optimal weights of the portfolio.

In [126]:
prices = prices.unstack(0).fillna(method='ffill')
index = prices.columns
weights = np.array([1/len(returns.columns)]*len(returns.columns))
def minimize_variance(weights):
    weights.reshape(-1, 1)
    returns = prices.pct_change().dropna()
    cov = returns.cov()
    portfolio_variance = float(np.dot(np.dot(weights.T, cov), weights))
    return portfolio_variance * 10000

def constraint1(x):
    return np.sum(x) - 1

con1 = {'type': 'eq', 'fun': constraint1}
constraints = [con1]
bnds = [(0, None) for _ in range(len(weights))]
result = minimize(minimize_variance, weights, method='SLSQP', bounds=bnds, constraints=constraints)

In the first code block, we print the result of our optimization. The output indicates that the optimization was successful. The 'fun' key in the output dictionary represents the minimum value of the objective function, which is the minimum variance of the portfolio that we have found. The values under the 'x' key are the optimal weights for each asset in our portfolio that minimize the variance.

Next, we create a dictionary that links each currency pair to its optimal weight in the portfolio. We use the `zip` function to combine the indices (the names of the currency pairs) and the optimal weights. The result is a dictionary where each currency pair is linked to its optimal weight.

Finally, we save this dictionary of optimal weights to a pickle file for future use. The `pickle` module is used to serialize and deserialize Python objects. By serializing our dictionary of optimal weights, we can easily load it from the pickle file at a later date. This can be particularly useful if the optimization process is computationally expensive in terms of time and resources because we won't have to re-run the optimization every time we need the optimal weights.

Overall, this code block represents the final step of our portfolio optimization process. By minimizing the portfolio variance, we have created a portfolio strategy that, according to Modern Portfolio Theory, should provide us with the most stable returns for a given level of risk.

In [127]:
result

 message: Optimization terminated successfully
 success: True
  status: 0
     fun: 0.002848667171112722
       x: [ 3.019e-01  0.000e+00  0.000e+00  3.227e-01  2.466e-01
            1.288e-01]
     nit: 15
     jac: [ 6.008e-03  1.128e-02  1.080e-02  5.310e-03  5.937e-03
            5.483e-03]
    nfev: 106
    njev: 15

In [128]:
dict(zip(index, list(result.x)))

{'EURGBP': 0.30189857553076915,
 'EURUSD': 0.0,
 'GBPJPY': 0.0,
 'GBPUSD': 0.32268924221792383,
 'USDCAD': 0.24659302521026463,
 'USDJPY': 0.12881915704104258}

In [131]:
import pickle
with open('weights.pkl', 'wb') as f:
    pickle.dump(dict(zip(index, list(result.x))), f)